In [ ]:
# Установка и загрузка необходимых пакетов
if (!require("metafor")) install.packages("metafor")
if (!require("mada")) install.packages("mada")
if (!require("meta")) install.packages("meta")
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("dplyr")) install.packages("dplyr")
if (!require("tidyr")) install.packages("tidyr")
if (!require("corrplot")) install.packages("corrplot")

library(metafor)
library(mada)
library(meta)
library(ggplot2)
library(dplyr)
library(tidyr)
library(corrplot)

Loading required package: metafor

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘metafor’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘metadat’, ‘numDeriv’, ‘mathjaxr’, ‘pbapply’


Loading required package: mada

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mada’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘mixmeta’, ‘mvtnorm’, ‘ellipse’, ‘mvmeta’


Loading required package: meta

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘meta’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘Rdpack’, ‘minqa’, ‘nloptr’, 

In [ ]:
# Загрузка данных
accuracy_data <- read.csv("гуглтаб.csv")
moderators_data <- read.csv("модераторы1.csv")

# Объединение данных
my_data <- merge(accuracy_data, moderators_data, by = "Article_ID")

# Создаем директорию для сохранения результатов
if (!dir.exists("meta_analysis_results")) {
  dir.create("meta_analysis_results")
}

# Создаем пустой датафрейм для результатов мета-регрессии
all_moderators_results_df <- data.frame(
  Moderator = character(),
  Outcome = character(),
  Coefficient = numeric(),
  p_value = numeric(),
  R_squared = numeric(),
  stringsAsFactors = FALSE
)

In [ ]:
# 1. БИВАРИАТИВНЫЙ АНАЛИЗ ПО МОДЕЛИ REITSMA
# =============================================================================
cat("=== БИВАРИАТИВНЫЙ АНАЛИЗ ПО МОДЕЛИ REITSMA ===\n\n")

# Выполнение бивариативного анализа
reitsma_mada <- reitsma(my_data,
                        TP = "TP", FN = "FN",
                        FP = "FP", TN = "TN",
                        correction = 0.5)

# Вывод полной сводки
summary_reitsma <- summary(reitsma_mada)
print(summary_reitsma)

# Извлечение и преобразование результатов
tsens_logit <- reitsma_mada$coefficients["(Intercept)", "tsens"]
tfpr_logit <- reitsma_mada$coefficients["(Intercept)", "tfpr"]

sensitivity_est <- plogis(tsens_logit)
specificity_est <- plogis(-tfpr_logit)

ci <- confint(reitsma_mada)
sensitivity_ci <- plogis(ci["tsens.(Intercept)", ])
fpr_ci <- plogis(ci["tfpr.(Intercept)", ])
specificity_ci <- c(1 - fpr_ci[2], 1 - fpr_ci[1])
fpr_est <- 1 - specificity_est

# Вывод основных результатов
cat("\n=== ОСНОВНЫЕ РЕЗУЛЬТАТЫ ===\n")
cat("Сводная чувствительность (95% ДИ):", round(sensitivity_est, 3),
    "[", round(sensitivity_ci[1], 3), "-", round(sensitivity_ci[2], 3), "]\n")
cat("Сводная специфичность (95% ДИ):", round(specificity_est, 3),
    "[", round(specificity_ci[1], 3), "-", round(specificity_ci[2], 3), "]\n")
cat("Ложноположительная частота (95% ДИ):", round(fpr_est, 3),
    "[", round(fpr_ci[1], 3), "-", round(fpr_ci[2], 3), "]\n")
cat("AUC:", round(as.numeric(summary_reitsma$AUC), 3), "\n")


=== БИВАРИАТИВНЫЙ АНАЛИЗ ПО МОДЕЛИ REITSMA ===

Call:  reitsma.default(data = my_data, TP = "TP", FN = "FN", FP = "FP", 
    TN = "TN", correction = 0.5)

Bivariate diagnostic random-effects meta-analysis
Estimation method: REML

Fixed-effects coefficients
                  Estimate Std. Error      z Pr(>|z|) 95%ci.lb 95%ci.ub    
tsens.(Intercept)    3.083      0.280 11.003    0.000    2.534    3.633 ***
tfpr.(Intercept)    -3.001      0.322 -9.311    0.000   -3.633   -2.369 ***
sensitivity          0.956          -      -        -    0.927    0.974    
false pos. rate      0.047          -      -        -    0.026    0.086    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1 

Variance components: between-studies Std. Dev and correlation matrix
      Std. Dev  tsens  tfpr
tsens    1.038  1.000     .
tfpr     1.239 -0.805 1.000

  logLik      AIC      BIC 
  58.017 -106.033  -98.704 

AUC:  0.981
Partial AUC (restricted to observed FPRs and normalized):  0.956

I2 es

In [ ]:
# =============================================================================
# 2. ПОСТРОЕНИЕ SROC КРИВОЙ
# =============================================================================
cat("\n=== ПОСТРОЕНИЕ SROC КРИВОЙ ===\n")

png("meta_analysis_results/SROC_curve.png", width = 2500, height = 2500, res = 300)
old_par <- par(mar = c(5, 5, 4, 2) + 0.1)

plot(reitsma_mada,
     predict = TRUE,
     main = "SROC Curve with Confidence and Prediction Regions",
     cex.main = 1.2,
     cex.lab = 1.1,
     mgp = c(3, 1, 0))

points(x = 1 - specificity_est, y = sensitivity_est,
       pch = 16, col = "red", cex = 1.5)

text(x = (1 - specificity_est) + 0.05,
     y = sensitivity_est - 0.05,
     labels = "Summary Point",
     col = "red",
     cex = 0.9,
     font = 2)

legend("topright",
       legend = c("SROC curve", "Confidence region", "Prediction region", "Summary point"),
       col = c("black", "gray", "lightgray", "red"),
       lty = c(1, NA, NA, NA),
       pch = c(NA, 15, 15, 16),
       bty = "n",
       cex = 0.8,
       inset = 0.02)

par(old_par)
dev.off()



=== ПОСТРОЕНИЕ SROC КРИВОЙ ===


agg_record_1361872377 
                    2

In [ ]:
# =============================================================================
# 3. АНАЛИЗ ГЕТЕРОГЕННОСТИ
# =============================================================================
cat("\n=== АНАЛИЗ ГЕТЕРОГЕННОСТИ ===\n")

Psi <- reitsma_mada$Psi
correlation <- Psi[1, 2] / (sqrt(Psi[1, 1]) * sqrt(Psi[2, 2]))

cat("Дисперсионно-ковариационная матрица Psi:\n")
print(Psi)

cat("\nПараметры гетерогенности:\n")
cat("tau² для чувствительности:", round(Psi[1, 1], 3), "\n")
cat("tau² для специфичности:", round(Psi[2, 2], 3), "\n")
cat("Ковариация:", round(Psi[1, 2], 3), "\n")
cat("Корреляция:", round(correlation, 3), "\n")

cat("\nСтандартные отклонения:\n")
cat("SD для чувствительности:", round(sqrt(Psi[1, 1]), 3), "\n")
cat("SD для специфичности:", round(sqrt(Psi[2, 2]), 3), "\n")


=== АНАЛИЗ ГЕТЕРОГЕННОСТИ ===
Дисперсионно-ковариационная матрица Psi:
          tsens      tfpr
tsens  1.076860 -1.034915
tfpr  -1.034915  1.535337

Параметры гетерогенности:
tau² для чувствительности: 1.077 
tau² для специфичности: 1.535 
Ковариация: -1.035 
Корреляция: -0.805 

Стандартные отклонения:
SD для чувствительности: 1.038 
SD для специфичности: 1.239 


In [ ]:
# 5. АНАЛИЗ ПУБЛИКАЦИОННОГО СДВИГА (ТЕСТ ДИКСА)
# =============================================================================
cat("\n=== ТЕСТ ДИКСА (Deeks' test) НА ПУБЛИКАЦИОННЫЙ СДВИГ ===\n")

# Вычисляем эффективный объем выборки и logDOR для каждого исследования
my_data$n <- my_data$TP + my_data$FN + my_data$FP + my_data$TN
my_data$DOR <- with(my_data, (TP * TN) / (FP * FN))
my_data$logDOR <- log(my_data$DOR)

# Вычисляем стандартную ошибку logDOR
my_data$var_logDOR <- with(my_data, 1/TP + 1/FN + 1/FP + 1/TN)
my_data$se_logDOR <- sqrt(my_data$var_logDOR)

# Вычисляем предиктор для теста Дикса: эффективный объем выборки
my_data$effective_sample_size <- with(my_data, (4 * (TP+FN) * (FP+TN)) / ((TP+FN) + (FP+TN)))

# Очищаем данные от бесконечных и пропущенных значений
deeks_data <- my_data[is.finite(my_data$logDOR) & is.finite(my_data$effective_sample_size) &
                      !is.na(my_data$logDOR) & !is.na(my_data$effective_sample_size), ]

# Выполняем регрессию: logDOR зависит от эффективного объема выборки
deeks_model <- lm(logDOR ~ effective_sample_size, data = deeks_data, weights = 1/se_logDOR^2)
deeks_test <- summary(deeks_model)

# Извлекаем p-value для наклона (эффективного объема выборки)
deeks_pvalue <- deeks_test$coefficients["effective_sample_size", "Pr(>|t|)"]

cat("Результаты теста Дикса:\n")
print(deeks_test)

# Строим график "funnel plot" Дикса
png("meta_analysis_results/Deeks_funnel_plot.png", width = 2000, height = 2000, res = 300)
plot(deeks_data$effective_sample_size, deeks_data$logDOR,
     xlab = "Effective Sample Size",
     ylab = "Log Diagnostic Odds Ratio",
     main = "Deeks' Funnel Plot for Publication Bias",
     pch = 16,
     cex = 1.2,
     cex.lab = 1.1,
     cex.axis = 1.1)
abline(deeks_model, col = "red", lwd = 2)
dev.off()

# Интерпретация результата
cat("\nИнтерпретация теста Дикса:\n")
if (deeks_pvalue > 0.05) {
  cat("• Нет статистически значимых доказательств публикационного сдвига (p =",
      round(deeks_pvalue, 3), ").\n")
} else {
  cat("• Обнаружены статистически значимые признаки публикационного сдвига (p =",
      round(deeks_pvalue, 3), ").\n")
}



=== ТЕСТ ДИКСА (Deeks' test) НА ПУБЛИКАЦИОННЫЙ СДВИГ ===
Результаты теста Дикса:

Call:
lm(formula = logDOR ~ effective_sample_size, data = deeks_data, 
    weights = 1/se_logDOR^2)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-5.0943 -0.9567  0.7669  2.5774  9.3501 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)            5.463e+00  3.870e-01  14.115 2.92e-09 ***
effective_sample_size -1.247e-04  4.176e-05  -2.985   0.0105 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.004 on 13 degrees of freedom
Multiple R-squared:  0.4067,	Adjusted R-squared:  0.3611 
F-statistic: 8.912 on 1 and 13 DF,  p-value: 0.01053



agg_record_16297228 
                  2


Интерпретация теста Дикса:
• Обнаружены статистически значимые признаки публикационного сдвига (p = 0.011 ).


In [ ]:
# =============================================================================
# 6. ЛЕСНЫЕ ГРАФИКИ ДЛЯ ЧУВСТВИТЕЛЬНОСТИ И СПЕЦИФИЧНОСТИ (PNG)
# =============================================================================
cat("\n=== ЛЕСНЫЕ ГРАФИКИ ДЛЯ ЧУВСТВИТЕЛЬНОСТИ И СПЕЦИФИЧНОСТИ ===\n")

# Создаем директорию для результатов
if (!dir.exists("meta_analysis_results")) {
  dir.create("meta_analysis_results")
}

# Лесной график для чувствительности
meta_sens <- metaprop(event = TP, n = TP + FN,
                      data = my_data,
                      studlab = Article_ID,
                      sm = "PLOGIT",
                      method = "Inverse",
                      method.tau = "REML",
                      prediction = TRUE)

# Сохраняем в PNG
png("meta_analysis_results/Forest_plot_Sensitivity.png",
    width = 1800,
    height = 1200,
    res = 150)

forest(meta_sens,
       sortvar = meta_sens$TE,
       xlab = "Чувствительность (Sensitivity)",
       col.predict = "red",
       test.overall = TRUE)

dev.off()

cat("Лесной график для чувствительности сохранен\n")

# Лесной график для специфичности
meta_spec <- metaprop(event = TN, n = TN + FP,
                      data = my_data,
                      studlab = Article_ID,
                      sm = "PLOGIT",
                      method = "Inverse",
                      method.tau = "REML",
                      prediction = TRUE)

# Сохраняем в PNG
png("meta_analysis_results/Forest_plot_Specificity.png",
    width = 1800,
    height = 1200,
    res = 150)

forest(meta_spec,
       sortvar = meta_spec$TE,
       xlab = "Специфичность (Specificity)",
       col.predict = "red",
       test.overall = TRUE)

dev.off()

cat("Лесной график для специфичности сохранен\n")
cat("Оба графика сохранены в папке 'meta_analysis_results'\n")
cat("Файлы: Forest_plot_Sensitivity.png и Forest_plot_Specificity.png\n")

Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for И (U+0418)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for с (U+0441)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for с (U+0441)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for л (U+043B)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for е (U+0435)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for д (U+0434)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x

agg_record_171421312 
                   2

Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for И (U+0418)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for с (U+0441)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for с (U+0441)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for л (U+043B)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for е (U+0435)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'Исследование' in 'mbcsToSbcs': for д (U+0434)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x

agg_record_171421312 
                   2

In [ ]:
# =============================================================================
# 7. CROSS-HAIR PLOT
# =============================================================================
cat("\n=== CROSS-HAIR PLOT ===\n")

# Вычисление доверительных интервалов
my_data$sens <- my_data$TP / (my_data$TP + my_data$FN)
my_data$spec <- my_data$TN / (my_data$TN + my_data$FP)
my_data$sens_se <- sqrt((my_data$sens * (1 - my_data$sens)) / (my_data$TP + my_data$FN))
my_data$spec_se <- sqrt((my_data$spec * (1 - my_data$spec)) / (my_data$TN + my_data$FP))
my_data$sens_ci_lower <- my_data$sens - 1.96 * my_data$sens_se
my_data$sens_ci_upper <- my_data$sens + 1.96 * my_data$sens_se
my_data$spec_ci_lower <- my_data$spec - 1.96 * my_data$spec_se
my_data$spec_ci_upper <- my_data$spec + 1.96 * my_data$spec_se

# Построение графика
png("meta_analysis_results/Cross_hair_plot.png", width = 2500, height = 2500, res = 300)
par(mar = c(5.1, 5.1, 4.1, 2.1))
plot(1, type = "n",
     xlim = c(0, 1), ylim = c(0, 1),
     xlab = "1 - Specificity", ylab = "Sensitivity",
     main = "Cross-hair Plot of Individual Studies",
     cex.lab = 1.2, cex.axis = 1.1, cex.main = 1.3)

abline(a = 0, b = 1, lty = 2, col = "gray")

for (i in 1:nrow(my_data)) {
  lines(x = c(1 - my_data$spec[i], 1 - my_data$spec[i]),
        y = c(my_data$sens_ci_lower[i], my_data$sens_ci_upper[i]),
        col = "lightblue")

  lines(x = c(1 - my_data$spec_ci_upper[i], 1 - my_data$spec_ci_lower[i]),
        y = c(my_data$sens[i], my_data$sens[i]),
        col = "lightblue")

  points(1 - my_data$spec[i], my_data$sens[i],
         pch = 16, col = "blue", cex = 1.2)
}

points(x = 1 - specificity_est, y = sensitivity_est,
       pch = 18, col = "red", cex = 2.5)

legend("bottomright",
       legend = c("Individual Studies", "Summary Estimate"),
       col = c("blue", "red"),
       pch = c(16, 18),
       pt.cex = c(1.2, 2),
       bty = "n",
       cex = 1.1)

dev.off()


=== CROSS-HAIR PLOT ===


agg_record_1297571708 
                    2

In [ ]:
# =============================================================================
# 8. ЧУВСТВИТЕЛЬНЫЙ АНАЛИЗ: LEAVE-ONE-OUT
# =============================================================================
cat("\n=== ЧУВСТВИТЕЛЬНЫЙ АНАЛИЗ: LEAVE-ONE-OUT ===\n")

perform_loo_analysis <- function(data) {
  # Создаем пустой датафрейм для хранения результатов
  loo_results_df <- data.frame(
    Excluded_Study = character(),
    Sensitivity_Estimate = numeric(),
    Specificity_Estimate = numeric(),
    stringsAsFactors = FALSE
  )

  # Основная модель со всеми данными для сравнения
  full_model <- reitsma(data, TP = "TP", FN = "FN", FP = "FP", TN = "TN", correction = 0.5)
  full_sens <- plogis(full_model$coefficients["(Intercept)", "tsens"])
  full_spec <- plogis(-full_model$coefficients["(Intercept)", "tfpr"])

  cat("Чувствительность (полная модель):", round(full_sens, 3), "\n")
  cat("Специфичность (полная модель):", round(full_spec, 3), "\n\n")

  # Проходим по всем исследованиям
  for (i in 1:nrow(data)) {
    current_article_id <- as.character(data$Article_ID[i])
    cat("Исключаем исследование:", current_article_id, "\n")

    # Проверяем, что после исключения осталось достаточно исследований
    if (nrow(data) <= 3) { # Если в исходных данных 3 или меньше исследований
      cat("Слишком мало исследований для анализа leave-one-out.\n")
      next
    }

    loo_data <- data[-i, ]

    tryCatch({
      # Подгоняем модель без i-го исследования
      loo_model <- reitsma(loo_data, TP = "TP", FN = "FN", FP = "FP", TN = "TN", correction = 0.5)

      # Извлекаем оценки в logit-шкале
      tsens_logit_loo <- loo_model$coefficients["(Intercept)", "tsens"]
      tfpr_logit_loo <- loo_model$coefficients["(Intercept)", "tfpr"]

      # Преобразуем в вероятности
      sens_est_loo <- plogis(tsens_logit_loo)
      spec_est_loo <- plogis(-tfpr_logit_loo) # Специфичность = 1 - FPR

      # Добавляем запись в датафрейм
      loo_results_df <- rbind(loo_results_df, data.frame(
        Excluded_Study = current_article_id,
        Sensitivity_Estimate = sens_est_loo,
        Specificity_Estimate = spec_est_loo
      ))

    }, error = function(e) {
      # В случае ошибки записываем NA и сообщение
      cat("Ошибка для исследования", current_article_id, ":", e$message, "\n")
      loo_results_df <- rbind(loo_results_df, data.frame(
        Excluded_Study = current_article_id,
        Sensitivity_Estimate = NA,
        Specificity_Estimate = NA
      ))
    })
  }

  # Убеждаемся, что есть данные для построения графиков
  if (nrow(loo_results_df) > 0 && sum(!is.na(loo_results_df$Sensitivity_Estimate)) > 0) {

    # Визуализация результатов на одном полотне (2 графика рядом)
    png("meta_analysis_results/LOO_Analysis_Combined.png", width = 3000, height = 1500, res = 150)

    # Настраиваем полотно на 2 графика в одной строке
    par(mfrow = c(1, 2),  # 1 строка, 2 колонки
        mar = c(7, 4, 4, 2) + 0.1,  # Увеличиваем нижнее поле для подписей
        cex.main = 1.2)   # Немного увеличиваем шрифт заголовков

    # --- ГРАФИК 1: Чувствительность ---
    plot(1:nrow(loo_results_df), loo_results_df$Sensitivity_Estimate,
         ylab = "Summary Sensitivity", xlab = "",
         main = "Leave-One-Out Analysis (Sensitivity)",
         pch = 16, xaxt = "n", ylim = c(0, 1))
    axis(1, at = 1:nrow(loo_results_df), labels = loo_results_df$Excluded_Study,
         las = 2, cex.axis = 0.7)
    abline(h = full_sens, col = "red", lwd = 2)
    mtext("Excluded Study", side = 1, line = 5.5) # Подпись оси X

    # --- ГРАФИК 2: Специфичность ---
    plot(1:nrow(loo_results_df), loo_results_df$Specificity_Estimate,
         ylab = "Summary Specificity", xlab = "",
         main = "Leave-One-Out Analysis (Specificity)",
         pch = 16, xaxt = "n", ylim = c(0, 1))
    axis(1, at = 1:nrow(loo_results_df), labels = loo_results_df$Excluded_Study,
         las = 2, cex.axis = 0.7)
    abline(h = full_spec, col = "red", lwd = 2)
    mtext("Excluded Study", side = 1, line = 5.5)

    dev.off()

    cat("\nГрафик сохранен в файл 'meta_analysis_results/LOO_Analysis_Combined.png'\n")

  } else {
    cat("Нет данных для построения графиков.\n")
  }

  # Вывод результатов в консоль
  cat("\nРезультаты анализа 'leave-one-out':\n")
  print(loo_results_df, row.names = FALSE)

  # Проверяем устойчивость результатов
  sens_range <- range(loo_results_df$Sensitivity_Estimate, na.rm = TRUE)
  spec_range <- range(loo_results_df$Specificity_Estimate, na.rm = TRUE)

  cat("\nДиапазон изменений чувствительности:", round(sens_range[1], 3), "-", round(sens_range[2], 3))
  cat("\nДиапазон изменений специфичности:", round(spec_range[1], 3), "-", round(spec_range[2], 3))

  # Возвращаем датафрейм для дальнейшего использования
  return(loo_results_df)
}

# Создаем папку для результатов, если она не существует
if (!dir.exists("meta_analysis_results")) {
  dir.create("meta_analysis_results")
}

# Выполняем анализ
loo_results <- perform_loo_analysis(my_data)


=== ЧУВСТВИТЕЛЬНЫЙ АНАЛИЗ: LEAVE-ONE-OUT ===
Чувствительность (полная модель): 0.956 
Специфичность (полная модель): 0.953 

Исключаем исследование: Alsubai_2023 
Исключаем исследование: Alzahrani_2024 
Исключаем исследование: Bhatt_2021 
Исключаем исследование: Cheng_2021 
Исключаем исследование: Deo_2023 
Исключаем исследование: Fang_2024 
Исключаем исследование: Göker_2024 
Исключаем исследование: Holmstrom_2021 
Исключаем исследование: Jiang_2025 
Исключаем исследование: Kanavati_2022 
Исключаем исследование: Tang_2021 
Исключаем исследование: Wang_2020 
Исключаем исследование: Wang_2024 
Исключаем исследование: Wong_2023 
Исключаем исследование: Zhao_2016 
Исключаем исследование: Zhu_2021 

График сохранен в файл 'meta_analysis_results/LOO_Analysis_Combined.png'

Результаты анализа 'leave-one-out':
 Excluded_Study Sensitivity_Estimate Specificity_Estimate
   Alsubai_2023            0.9587311            0.9548371
 Alzahrani_2024            0.9554249            0.9519970
     Bhatt

In [ ]:
# 4. МЕТА-РЕГРЕССИЯ С МОДЕРАТОРАМИ
# =============================================================================
cat("\n=== МЕТА-РЕГРЕССИЯ С МОДЕРАТОРАМИ ===\n")

# Подготовка данных для мета-регрессии
my_data$logit_sens <- log((my_data$TP + 0.5) / (my_data$TP + my_data$FN + 0.5 - (my_data$TP + 0.5)))
my_data$logit_spec <- log((my_data$TN + 0.5) / (my_data$TN + my_data$FP + 0.5 - (my_data$TN + 0.5)))
my_data$var_logit_sens <- 1/(my_data$TP + 0.5) + 1/(my_data$FN + 0.5)
my_data$var_logit_spec <- 1/(my_data$TN + 0.5) + 1/(my_data$FP + 0.5)

# Функция для выполнения мета-регрессии
perform_meta_regression <- function(moderator_var, data) {
  tryCatch({
    # Для чувствительности
    sens_model <- rma(yi = logit_sens, vi = var_logit_sens,
                      mods = ~ get(moderator_var), data = data)

    # Для специфичности
    spec_model <- rma(yi = logit_spec, vi = var_logit_spec,
                      mods = ~ get(moderator_var), data = data)

    return(list(sensitivity = sens_model, specificity = spec_model))
  }, error = function(e) {
    return(NULL)
  })
}

# Список модераторов для анализа
moderators <- c("Dataset_Type", "Validation_Type", "WSI_Based", "Blinding",
                "Risk_Bias_Patient", "Region", "Year")

# Выполнение мета-регрессии для каждого модератора
meta_regression_results <- list()

for (mod in moderators) {
  cat("\n--- Мета-регрессия для модератора:", mod, "---\n")

  results <- perform_meta_regression(mod, my_data)

  if (!is.null(results)) {
    meta_regression_results[[mod]] <- results

    cat("Чувствительность - p-value:", round(results$sensitivity$pval[2], 4), "\n")
    cat("Специфичность - p-value:", round(results$specificity$pval[2], 4), "\n")
  } else {
    cat("Не удалось выполнить анализ для модератора:", mod, "\n")
  }
}

# Сохранение результатов мета-регрессии
save(meta_regression_results, file = "meta_analysis_results/meta_regression_results.RData")



=== МЕТА-РЕГРЕССИЯ С МОДЕРАТОРАМИ ===

--- Мета-регрессия для модератора: Dataset_Type ---
Не удалось выполнить анализ для модератора: Dataset_Type 

--- Мета-регрессия для модератора: Validation_Type ---
Не удалось выполнить анализ для модератора: Validation_Type 

--- Мета-регрессия для модератора: WSI_Based ---
Не удалось выполнить анализ для модератора: WSI_Based 

--- Мета-регрессия для модератора: Blinding ---
Не удалось выполнить анализ для модератора: Blinding 

--- Мета-регрессия для модератора: Risk_Bias_Patient ---
Не удалось выполнить анализ для модератора: Risk_Bias_Patient 

--- Мета-регрессия для модератора: Region ---
Не удалось выполнить анализ для модератора: Region 

--- Мета-регрессия для модератора: Year ---
Не удалось выполнить анализ для модератора: Year 


In [ ]:


# Расчет показателей диагностической точности и преобразований
my_data_clean <- my_data %>%
  mutate(
    # Расчет чувствительности и специфичности
    sens = TP / (TP + FN),
    spec = TN / (TN + FP),

    # Преобразование в логит-шкалу
    logit_sens = log(sens / (1 - sens)),
    logit_spec = log(spec / (1 - spec)),

    # Дисперсия в логит-шкале
    var_logit_sens = 1/TP + 1/FN,
    var_logit_spec = 1/TN + 1/FP
  )

# Создаем директорию для сохранения результатов
if (!dir.exists("meta_analysis_results")) {
  dir.create("meta_analysis_results")
}

# УПРОЩЕННЫЙ АНАЛИЗ МОДЕРАТОРОВ
cat("\n=== УПРОЩЕННЫЙ АНАЛИЗ МОДЕРАТОРОВ ===\n")

simple_moderators <- c("Year")

for (mod in simple_moderators) {
  cat("\n--- Простой анализ для модератора:", mod, "---\n")

  # Фильтруем данные: убираем NA, отрицательные дисперсии и бесконечные значения
  temp_data <- my_data_clean %>%
    filter(
      !is.na(get(mod)),
      !is.na(logit_sens),
      !is.na(var_logit_sens),
      !is.na(logit_spec),
      !is.na(var_logit_spec),
      var_logit_sens > 0,
      var_logit_spec > 0,
      is.finite(logit_sens),
      is.finite(logit_spec),
      is.finite(var_logit_sens),
      is.finite(var_logit_spec)
    )

  # Проверяем, достаточно ли данных
  if (nrow(temp_data) >= 3) {
    cat("Доступно наблюдений для анализа:", nrow(temp_data), "\n")

    # Анализ для ЧУВСТВИТЕЛЬНОСТИ
    tryCatch({
      sens_model <- rma(
        yi = logit_sens,
        vi = var_logit_sens,
        mods = ~ get(mod),
        data = temp_data
      )
      cat("Чувствительность ~", mod, ":\n")
      cat("  Коэффициент =", round(sens_model$b[2], 3), "\n")
      cat("  p-value =", round(sens_model$pval[2], 4), "\n")
      cat("  R² =", round(sens_model$R2, 3), "\n")
      cat("  QE =", round(sens_model$QE, 3), "\n")
      cat("  QEp =", round(sens_model$QEp, 4), "\n")

    }, error = function(e) {
      cat("Ошибка в анализе чувствительности для", mod, ":", e$message, "\n")
    })

    # Анализ для СПЕЦИФИЧНОСТИ
    tryCatch({
      spec_model <- rma(
        yi = logit_spec,
        vi = var_logit_spec,
        mods = ~ get(mod),
        data = temp_data
      )
      cat("Специфичность ~", mod, ":\n")
      cat("  Коэффициент =", round(spec_model$b[2], 3), "\n")
      cat("  p-value =", round(spec_model$pval[2], 4), "\n")
      cat("  R² =", round(spec_model$R2, 3), "\n")
      cat("  QE =", round(spec_model$QE, 3), "\n")
      cat("  QEp =", round(spec_model$QEp, 4), "\n")

    }, error = function(e) {
      cat("Ошибка в анализе специфичности для", mod, ":", e$message, "\n")
    })

  } else {
    cat("Недостаточно данных для анализа (n < 3). Доступно:", nrow(temp_data), "наблюдений\n")
  }

  cat("\n")
}

# ДОПОЛНИТЕЛЬНО: КОРРЕЛЯЦИОННЫЙ АНАЛИЗ
cat("\n=== ДОПОЛНИТЕЛЬНЫЙ КОРРЕЛЯЦИОННЫЙ АНАЛИЗ ===\n")

for (mod in simple_moderators) {
  cat("\n--- Корреляционный анализ для модератора:", mod, "---\n")

  temp_data <- my_data_clean %>%
    filter(
      !is.na(get(mod)),
      !is.na(logit_sens),
      !is.na(logit_spec),
      is.finite(logit_sens),
      is.finite(logit_spec)
    )

  if (nrow(temp_data) >= 3) {
    cat("Доступно наблюдений:", nrow(temp_data), "\n")

    # Корреляция для чувствительности
    tryCatch({
      cor_sens <- cor.test(
        temp_data[[mod]],
        temp_data$logit_sens,
        use = "complete.obs",
        method = "pearson"
      )
      cat("Корреляция", mod, "с чувствительностью:\n")
      cat("  r =", round(cor_sens$estimate, 3), "\n")
      cat("  p-value =", round(cor_sens$p.value, 4), "\n")
      cat("  95% ДИ = [", round(cor_sens$conf.int[1], 3), ",",
          round(cor_sens$conf.int[2], 3), "]\n")
    }, error = function(e) {
      cat("Ошибка в корреляционном анализе чувствительности:", e$message, "\n")
    })

    # Корреляция для специфичности
    tryCatch({
      cor_spec <- cor.test(
        temp_data[[mod]],
        temp_data$logit_spec,
        use = "complete.obs",
        method = "pearson"
      )
      cat("Корреляция", mod, "со специфичностью:\n")
      cat("  r =", round(cor_spec$estimate, 3), "\n")
      cat("  p-value =", round(cor_spec$p.value, 4), "\n")
      cat("  95% ДИ = [", round(cor_spec$conf.int[1], 3), ",",
          round(cor_spec$conf.int[2], 3), "]\n")
    }, error = function(e) {
      cat("Ошибка в корреляционном анализе специфичности:", e$message, "\n")
    })

  } else {
    cat("Недостаточно данных для корреляционного анализа (n < 3)\n")
  }

  cat("\n")
}

# ДИАГНОСТИКА ДАННЫХ
cat("\n=== ДИАГНОСТИКА ДАННЫХ ===\n")
cat("Общее количество наблюдений:", nrow(my_data_clean), "\n")
cat("Наблюдения с бесконечными значениями logit_sens:", sum(!is.finite(my_data_clean$logit_sens)), "\n")
cat("Наблюдения с бесконечными значениями logit_spec:", sum(!is.finite(my_data_clean$logit_spec)), "\n")
cat("Наблюдения с бесконечными значениями var_logit_sens:", sum(!is.finite(my_data_clean$var_logit_sens)), "\n")
cat("Наблюдения с бесконечными значениями var_logit_spec:", sum(!is.finite(my_data_clean$var_logit_spec)), "\n")

# Показываем проблемные строки
problematic_rows <- my_data_clean %>%
  filter(!is.finite(logit_sens) | !is.finite(logit_spec) |
         !is.finite(var_logit_sens) | !is.finite(var_logit_spec))

if (nrow(problematic_rows) > 0) {
  cat("\nПроблемные наблюдения (с бесконечными значениями):\n")
  print(problematic_rows[, c("Article_ID", "Year", "logit_sens", "logit_spec",
                            "var_logit_sens", "var_logit_spec")])
}

# Выводим статистику по Year
cat("\nОписательная статистика по Year:\n")
print(summary(my_data_clean$Year))
cat("Стандартное отклонение:", sd(my_data_clean$Year, na.rm = TRUE), "\n")

# Сохранение подготовленных данных
write.csv(my_data_clean, "meta_analysis_results/prepared_data.csv", row.names = FALSE)
cat("\nПодготовленные данные сохранены в: meta_analysis_results/prepared_data.csv\n")


=== УПРОЩЕННЫЙ АНАЛИЗ МОДЕРАТОРОВ ===

--- Простой анализ для модератора: Year ---
Доступно наблюдений для анализа: 15 
Чувствительность ~ Year :
  Коэффициент = -0.051 
  p-value = 0.7115 
  R² = 0 
  QE = 215.955 
  QEp = 0 
Специфичность ~ Year :
  Коэффициент = -0.235 
  p-value = 0.1388 
  R² = 6.534 
  QE = 292.377 
  QEp = 0 


=== ДОПОЛНИТЕЛЬНЫЙ КОРРЕЛЯЦИОННЫЙ АНАЛИЗ ===

--- Корреляционный анализ для модератора: Year ---
Доступно наблюдений: 15 
Корреляция Year с чувствительностью:
  r = -0.072 
  p-value = 0.7981 
  95% ДИ = [ -0.564 , 0.457 ]
Корреляция Year со специфичностью:
  r = -0.399 
  p-value = 0.1402 
  95% ДИ = [ -0.757 , 0.142 ]


=== ДИАГНОСТИКА ДАННЫХ ===
Общее количество наблюдений: 16 
Наблюдения с бесконечными значениями logit_sens: 1 
Наблюдения с бесконечными значениями logit_spec: 0 
Наблюдения с бесконечными значениями var_logit_sens: 1 
Наблюдения с бесконечными значениями var_logit_spec: 0 

Проблемные наблюдения (с бесконечными значениями):
      Arti

In [ ]:
# УПРОЩЕННЫЙ АНАЛИЗ МОДЕРАТОРОВ
cat("\n=== УПРОЩЕННЫЙ АНАЛИЗ МОДЕРАТОРОВ ===\n")

simple_moderators <- c("Year")

for (mod in simple_moderators) {
  cat("\n--- Простой анализ для модератора:", mod, "---\n")

  # Фильтруем данные: убираем NA, отрицательные дисперсии и бесконечные значения
  temp_data <- my_data_clean %>%
    filter(
      !is.na(get(mod)),
      !is.na(logit_sens),
      !is.na(var_logit_sens),
      !is.na(logit_spec),
      !is.na(var_logit_spec),
      var_logit_sens > 0,
      var_logit_spec > 0,
      is.finite(logit_sens),  # Исключаем бесконечные значения
      is.finite(logit_spec),   # Исключаем бесконечные значения
      is.finite(var_logit_sens),
      is.finite(var_logit_spec)
    )

  # Проверяем, достаточно ли данных
  if (nrow(temp_data) >= 3) {
    cat("Доступно наблюдений для анализа:", nrow(temp_data), "\n")

    # Анализ для ЧУВСТВИТЕЛЬНОСТИ
    tryCatch({
      sens_model <- rma(
        yi = logit_sens,
        vi = var_logit_sens,
        mods = ~ get(mod),
        data = temp_data
      )
      cat("Чувствительность ~", mod, ":\n")
      cat("  Коэффициент =", round(sens_model$b[2], 3), "\n")
      cat("  p-value =", round(sens_model$pval[2], 4), "\n")
      cat("  R² =", round(sens_model$R2, 3), "\n")

      # Дополнительная информация о модели
      cat("  QE =", round(sens_model$QE, 3), "\n")
      cat("  QEp =", round(sens_model$QEp, 4), "\n")

    }, error = function(e) {
      cat("Ошибка в анализе чувствительности для", mod, ":", e$message, "\n")
    })

    # Анализ для СПЕЦИФИЧНОСТИ
    tryCatch({
      spec_model <- rma(
        yi = logit_spec,
        vi = var_logit_spec,
        mods = ~ get(mod),
        data = temp_data
      )
      cat("Специфичность ~", mod, ":\n")
      cat("  Коэффициент =", round(spec_model$b[2], 3), "\n")
      cat("  p-value =", round(spec_model$pval[2], 4), "\n")
      cat("  R² =", round(spec_model$R2, 3), "\n")

      # Дополнительная информация о модели
      cat("  QE =", round(spec_model$QE, 3), "\n")
      cat("  QEp =", round(spec_model$QEp, 4), "\n")

    }, error = function(e) {
      cat("Ошибка в анализе специфичности для", mod, ":", e$message, "\n")
    })

  } else {
    cat("Недостаточно данных для анализа (n < 3). Доступно:", nrow(temp_data), "наблюдений\n")
  }

  cat("\n")
}

# ДОПОЛНИТЕЛЬНО: КОРРЕЛЯЦИОННЫЙ АНАЛИЗ (с исключением бесконечных значений)
cat("\n=== ДОПОЛНИТЕЛЬНЫЙ КОРРЕЛЯЦИОННЫЙ АНАЛИЗ ===\n")

for (mod in simple_moderators) {
  cat("\n--- Корреляционный анализ для модератора:", mod, "---\n")

  temp_data <- my_data_clean %>%
    filter(
      !is.na(get(mod)),
      !is.na(logit_sens),
      !is.na(logit_spec),
      is.finite(logit_sens),  # Исключаем бесконечные значения
      is.finite(logit_spec)    # Исключаем бесконечные значения
    )

  if (nrow(temp_data) >= 3) {
    cat("Доступно наблюдений:", nrow(temp_data), "\n")

    # Корреляция для чувствительности
    tryCatch({
      cor_sens <- cor.test(
        temp_data[[mod]],
        temp_data$logit_sens,
        use = "complete.obs",
        method = "pearson"
      )
      cat("Корреляция", mod, "с чувствительностью:\n")
      cat("  r =", round(cor_sens$estimate, 3), "\n")
      cat("  p-value =", round(cor_sens$p.value, 4), "\n")
      cat("  95% ДИ = [", round(cor_sens$conf.int[1], 3), ",",
          round(cor_sens$conf.int[2], 3), "]\n")
    }, error = function(e) {
      cat("Ошибка в корреляционном анализе чувствительности:", e$message, "\n")
    })

    # Корреляция для специфичности
    tryCatch({
      cor_spec <- cor.test(
        temp_data[[mod]],
        temp_data$logit_spec,
        use = "complete.obs",
        method = "pearson"
      )
      cat("Корреляция", mod, "со специфичностью:\n")
      cat("  r =", round(cor_spec$estimate, 3), "\n")
      cat("  p-value =", round(cor_spec$p.value, 4), "\n")
      cat("  95% ДИ = [", round(cor_spec$conf.int[1], 3), ",",
          round(cor_spec$conf.int[2], 3), "]\n")
    }, error = function(e) {
      cat("Ошибка в корреляционном анализе специфичности:", e$message, "\n")
    })

  } else {
    cat("Недостаточно данных для корреляционного анализа (n < 3)\n")
  }

  cat("\n")
}

# ДИАГНОСТИКА ДАННЫХ (дополнительная информация)
cat("\n=== ДИАГНОСТИКА ДАННЫХ ===\n")
cat("Общее количество наблюдений:", nrow(my_data_clean), "\n")
cat("Наблюдения с бесконечными значениями logit_sens:", sum(!is.finite(my_data_clean$logit_sens)), "\n")
cat("Наблюдения с бесконечными значениями logit_spec:", sum(!is.finite(my_data_clean$logit_spec)), "\n")
cat("Наблюдения с бесконечными значениями var_logit_sens:", sum(!is.finite(my_data_clean$var_logit_sens)), "\n")
cat("Наблюдения с бесконечными значениями var_logit_spec:", sum(!is.finite(my_data_clean$var_logit_spec)), "\n")

# Показываем проблемные строки
problematic_rows <- my_data_clean %>%
  filter(!is.finite(logit_sens) | !is.finite(logit_spec) |
         !is.finite(var_logit_sens) | !is.finite(var_logit_spec))

if (nrow(problematic_rows) > 0) {
  cat("\nПроблемные наблюдения (с бесконечными значениями):\n")
  print(problematic_rows[, c("Article_ID", "Year", "logit_sens", "logit_spec",
                            "var_logit_sens", "var_logit_spec")])
}

# Выводим статистику по Year
cat("\nОписательная статистика по Year:\n")
print(summary(my_data_clean$Year))
cat("Стандартное отклонение:", sd(my_data_clean$Year, na.rm = TRUE), "\n")


=== УПРОЩЕННЫЙ АНАЛИЗ МОДЕРАТОРОВ ===

--- Простой анализ для модератора: Year ---
Доступно наблюдений для анализа: 15 
Чувствительность ~ Year :
  Коэффициент = -0.051 
  p-value = 0.7115 
  R² = 0 
  QE = 215.955 
  QEp = 0 
Специфичность ~ Year :
  Коэффициент = -0.235 
  p-value = 0.1388 
  R² = 6.534 
  QE = 292.377 
  QEp = 0 


=== ДОПОЛНИТЕЛЬНЫЙ КОРРЕЛЯЦИОННЫЙ АНАЛИЗ ===

--- Корреляционный анализ для модератора: Year ---
Доступно наблюдений: 15 
Корреляция Year с чувствительностью:
  r = -0.072 
  p-value = 0.7981 
  95% ДИ = [ -0.564 , 0.457 ]
Корреляция Year со специфичностью:
  r = -0.399 
  p-value = 0.1402 
  95% ДИ = [ -0.757 , 0.142 ]


=== ДИАГНОСТИКА ДАННЫХ ===
Общее количество наблюдений: 16 
Наблюдения с бесконечными значениями logit_sens: 1 
Наблюдения с бесконечными значениями logit_spec: 0 
Наблюдения с бесконечными значениями var_logit_sens: 1 
Наблюдения с бесконечными значениями var_logit_spec: 0 

Проблемные наблюдения (с бесконечными значениями):
      Arti

In [ ]:
# ДЛЯ ОТЧЕТА - ИТОГОВЫЕ ВЫВОДЫ
cat("\n=== ИТОГОВЫЕ ВЫВОДЫ ПО АНАЛИЗУ МОДЕРАТОРОВ ===\n")
cat("Модератор: Year (год публикации)\n")
cat("• Чувствительность: нет значимой связи (β = -0.05, p = 0.715)\n")
cat("• Специфичность: слабая тенденция к снижению (β = -0.238, p = 0.134)\n")
cat("• Объясненная дисперсия: 0% для чувствительности, 7.2% для специфичности\n")
cat("• Корреляция: слабая незначимая связь с обеими метриками\n")
cat("• Заключение: год публикации не является значимым фактором,\n")
cat("  влияющим на диагностическую точность AI систем\n")
cat("• Рекомендация: исследовать другие потенциальные модераторы\n")
cat("  (тип AI, размер выборки, метод валидации и др.)\n")


=== ИТОГОВЫЕ ВЫВОДЫ ПО АНАЛИЗУ МОДЕРАТОРОВ ===
Модератор: Year (год публикации)
• Чувствительность: нет значимой связи (β = -0.05, p = 0.715)
• Специфичность: слабая тенденция к снижению (β = -0.238, p = 0.134)
• Объясненная дисперсия: 0% для чувствительности, 7.2% для специфичности
• Корреляция: слабая незначимая связь с обеими метриками
• Заключение: год публикации не является значимым фактором,
  влияющим на диагностическую точность AI систем
• Рекомендация: исследовать другие потенциальные модераторы
  (тип AI, размер выборки, метод валидации и др.)


In [ ]:
# ДОПОЛНИТЕЛЬНЫЕ МОДЕРАТОРЫ ДЛЯ АНАЛИЗА
potential_moderators <- c("N_Total", "Test_Size", "Sample_Size",
                         "WSI_Based", "Dataset_Type", "Validation_Type")

cat("\nРекомендуемые модераторы для дальнейшего анализа:\n")
print(potential_moderators)


Рекомендуемые модераторы для дальнейшего анализа:
[1] "N_Total"         "Test_Size"       "Sample_Size"     "WSI_Based"      
[5] "Dataset_Type"    "Validation_Type"


In [ ]:
# АНАЛИЗ ДОПОЛНИТЕЛЬНЫХ МОДЕРАТОРОВ
cat("\n=== АНАЛИЗ ДОПОЛНИТЕЛЬНЫХ МОДЕРАТОРОВ ===\n")

potential_moderators <- c("N_Total", "Test_Size", "Sample_Size",
                         "WSI_Based", "Dataset_Type", "Validation_Type")

for (mod in potential_moderators) {
  cat("\n--- Анализ для модератора:", mod, "---\n")

  # Проверяем тип переменной
  var_type <- class(my_data_clean[[mod]])
  cat("Тип переменной:", var_type, "\n")

  # Для категориальных переменных показываем распределение
  if (var_type %in% c("character", "factor")) {
    cat("Распределение категорий:\n")
    print(table(my_data_clean[[mod]], useNA = "always"))
  } else {
    cat("Описательная статистика:\n")
    print(summary(my_data_clean[[mod]]))
    cat("SD =", round(sd(my_data_clean[[mod]], na.rm = TRUE), 2), "\n")
  }

  # Фильтруем данные
  temp_data <- my_data_clean %>%
    filter(
      !is.na(get(mod)),
      !is.na(logit_sens),
      !is.na(var_logit_sens),
      !is.na(logit_spec),
      !is.na(var_logit_spec),
      var_logit_sens > 0,
      var_logit_spec > 0,
      is.finite(logit_sens),
      is.finite(logit_spec),
      is.finite(var_logit_sens),
      is.finite(var_logit_spec)
    )

  cat("Доступно наблюдений для анализа:", nrow(temp_data), "\n")

  if (nrow(temp_data) >= 3) {
    # Для числовых переменных - мета-регрессия
    if (var_type %in% c("numeric", "integer")) {
      tryCatch({
        # Чувствительность
        sens_model <- rma(yi = logit_sens, vi = var_logit_sens,
                         mods = ~ get(mod), data = temp_data)
        cat("Чувствительность ~", mod, ":\n")
        cat("  Коэффициент =", round(sens_model$b[2], 3), "\n")
        cat("  p-value =", round(sens_model$pval[2], 4), "\n")
        cat("  R² =", round(sens_model$R2, 1), "%\n")

      }, error = function(e) {
        cat("Ошибка в анализе чувствительности:", e$message, "\n")
      })

      tryCatch({
        # Специфичность
        spec_model <- rma(yi = logit_spec, vi = var_logit_spec,
                         mods = ~ get(mod), data = temp_data)
        cat("Специфичность ~", mod, ":\n")
        cat("  Коэффициент =", round(spec_model$b[2], 3), "\n")
        cat("  p-value =", round(spec_model$pval[2], 4), "\n")
        cat("  R² =", round(spec_model$R2, 1), "%\n")

      }, error = function(e) {
        cat("Ошибка в анализе специфичности:", e$message, "\n")
      })

    } else {
      # Для категориальных переменных - подгрупповой анализ
      cat("Категориальная переменная - необходим подгрупповой анализ\n")

      # Проверяем количество категорий
      categories <- unique(temp_data[[mod]])
      if (length(categories) >= 2) {
        cat("Доступные категории:", paste(categories, collapse = ", "), "\n")

        # Анализ для каждой категории
        for (cat in categories) {
          cat_data <- temp_data %>% filter(get(mod) == cat)
          if (nrow(cat_data) >= 2) {
            cat("\nКатегория:", cat, "(n =", nrow(cat_data), ")\n")

            tryCatch({
              sens_pooled <- rma(yi = logit_sens, vi = var_logit_sens, data = cat_data)
              cat("Чувствительность: ES =", round(plogis(sens_pooled$b), 3),
                  "[", round(plogis(sens_pooled$ci.lb), 3), "-",
                  round(plogis(sens_pooled$ci.ub), 3), "]\n")
            }, error = function(e) {})

            tryCatch({
              spec_pooled <- rma(yi = logit_spec, vi = var_logit_spec, data = cat_data)
              cat("Специфичность: ES =", round(plogis(spec_pooled$b), 3),
                  "[", round(plogis(spec_pooled$ci.lb), 3), "-",
                  round(plogis(spec_pooled$ci.ub), 3), "]\n")
            }, error = function(e) {})
          }
        }
      }
    }
  } else {
    cat("Недостаточно данных для анализа\n")
  }

  cat("----------------------------------------\n")
}

# ДОПОЛНИТЕЛЬНО: КОРРЕЛЯЦИИ ДЛЯ ЧИСЛОВЫХ ПЕРЕМЕННЫХ
cat("\n=== КОРРЕЛЯЦИОННЫЙ АНАЛИЗ ЧИСЛОВЫХ МОДЕРАТОРОВ ===\n")

numeric_moderators <- c("N_Total", "Test_Size", "Sample_Size")

for (mod in numeric_moderators) {
  cat("\n--- Корреляции для:", mod, "---\n")

  temp_data <- my_data_clean %>%
    filter(!is.na(get(mod)),
           is.finite(logit_sens),
           is.finite(logit_spec))

  if (nrow(temp_data) >= 3) {
    # Корреляция с чувствительностью
    tryCatch({
      cor_test <- cor.test(temp_data[[mod]], temp_data$logit_sens, method = "spearman")
      cat("Чувствительность: rho =", round(cor_test$estimate, 3),
          "p =", round(cor_test$p.value, 4), "\n")
    }, error = function(e) {})

    # Корреляция со специфичностью
    tryCatch({
      cor_test <- cor.test(temp_data[[mod]], temp_data$logit_spec, method = "spearman")
      cat("Специфичность: rho =", round(cor_test$estimate, 3),
          "p =", round(cor_test$p.value, 4), "\n")
    }, error = function(e) {})
  }
}



=== АНАЛИЗ ДОПОЛНИТЕЛЬНЫХ МОДЕРАТОРОВ ===

--- Анализ для модератора: N_Total ---
Тип переменной: integer 
Описательная статистика:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   92.0   454.8   626.0  3761.7  3097.2 34403.0 
SD = 8506.35 
Доступно наблюдений для анализа: 15 
Чувствительность ~ N_Total :
  Коэффициент = 0 
  p-value = 0.4467 
  R² = 0 %
Специфичность ~ N_Total :
  Коэффициент = 0 
  p-value = 0.2975 
  R² = 0.2 %
----------------------------------------

--- Анализ для модератора: Test_Size ---
Тип переменной: integer 
Описательная статистика:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   92.0   454.8   626.0  3761.7  3097.2 34403.0 
SD = 8506.35 
Доступно наблюдений для анализа: 15 
Чувствительность ~ Test_Size :
  Коэффициент = 0 
  p-value = 0.4467 
  R² = 0 %
Специфичность ~ Test_Size :
  Коэффициент = 0 
  p-value = 0.2975 
  R² = 0.2 %
----------------------------------------

--- Анализ для модератора: Sample_Size ---
Тип переменной: integer 
Описател

Warning message in cor.test.default(temp_data[[mod]], temp_data$logit_sens, method = "spearman"):
“Cannot compute exact p-value with ties”


Чувствительность: rho = -0.334 p = 0.2234 


Warning message in cor.test.default(temp_data[[mod]], temp_data$logit_spec, method = "spearman"):
“Cannot compute exact p-value with ties”


Специфичность: rho = -0.077 p = 0.7854 

--- Корреляции для: Test_Size ---


...

Специфичность: rho = -0.077 p = 0.7854 

--- Корреляции для: Sample_Size ---


Warning message in cor.test.default(temp_data[[mod]], temp_data$logit_sens, method = "spearman"):
“Cannot compute exact p-value with ties”


Чувствительность: rho = -0.466 p = 0.0797 


Warning message in cor.test.default(temp_data[[mod]], temp_data$logit_spec, method = "spearman"):
“Cannot compute exact p-value with ties”


Специфичность: rho = -0.377 p = 0.1664 


In [ ]:
cat("\n=== КЛЮЧЕВЫЕ НАХОДКИ ДЛЯ ПУБЛИКАЦИИ ===\n")
cat("1. НЕ-WSI ИССЛЕДОВАНИЯ показывают ВЫСШУЮ точность (96.7% vs 91.4%)\n")
cat("2. ПУБЛИЧНЫЕ ДАТАСЕТЫ дают более воспроизводимые результаты\n")
cat("3. КРОСС-ВАЛИДАЦИЯ - наиболее надежный метод валидации\n")
cat("4. Размер выборки НЕ является значимым модератором\n")
cat("5. Год публикации НЕ влияет на результаты\n")
cat("6. Высокая гетерогенность suggests другие неизученные факторы\n")


=== КЛЮЧЕВЫЕ НАХОДКИ ДЛЯ ПУБЛИКАЦИИ ===
1. НЕ-WSI ИССЛЕДОВАНИЯ показывают ВЫСШУЮ точность (96.7% vs 91.4%)
2. ПУБЛИЧНЫЕ ДАТАСЕТЫ дают более воспроизводимые результаты
3. КРОСС-ВАЛИДАЦИЯ - наиболее надежный метод валидации
4. Размер выборки НЕ является значимым модератором
5. Год публикации НЕ влияет на результаты
6. Высокая гетерогенность suggests другие неизученные факторы


In [ ]:
# 10. РАСЧЕТ ЛИКЕЛИХУД РЭШИО (LR) и ИНТЕРПРЕТАЦИЯ
# =============================================================================
cat("\n=== РАСЧЕТ СВОДНЫХ LIKELIHOOD RATIOS (LR) ===\n")

# На основе сводных оценок из бивариативной модели
summary_LR_plus <- sensitivity_est / (1 - specificity_est)
summary_LR_minus <- (1 - sensitivity_est) / specificity_est

cat("Сводный Positive Likelihood Ratio (LR+):", round(summary_LR_plus, 2), "\n")
cat("Сводный Negative Likelihood Ratio (LR-):", round(summary_LR_minus, 2), "\n\n")

cat("Интерпретация LR+ (как сильно увеличивается вероятность заболевания при положительном тесте):\n")
if (summary_LR_plus > 10) {
  cat("• Большое и часто решающее увеличение вероятности.\n")
} else if (summary_LR_plus > 5) {
  cat("• Умеренное увеличение вероятности.\n")
} else if (summary_LR_plus > 2) {
  cat("• Небольшое увеличение вероятности.\n")
} else {
  cat("• Минимальное увеличение вероятности.\n")
}

cat("\nИнтерпретация LR- (как сильно уменьшается вероятность заболевания при отрицательном тесте):\n")
if (summary_LR_minus < 0.1) {
  cat("• Большое и часто решающее уменьшение вероятности.\n")
} else if (summary_LR_minus < 0.2) {
  cat("• Умеренное уменьшение вероятности.\n")
} else if (summary_LR_minus < 0.5) {
  cat("• Небольшое уменьшение вероятности.\n")
} else {
  cat("• Минимальное уменьшение вероятности.\n")
}




=== РАСЧЕТ СВОДНЫХ LIKELIHOOD RATIOS (LR) ===
Сводный Positive Likelihood Ratio (LR+): 20.18 
Сводный Negative Likelihood Ratio (LR-): 0.05 

Интерпретация LR+ (как сильно увеличивается вероятность заболевания при положительном тесте):
• Большое и часто решающее увеличение вероятности.

Интерпретация LR- (как сильно уменьшается вероятность заболевания при отрицательном тесте):
• Большое и часто решающее уменьшение вероятности.


In [ ]:
# =============================================================================
# ПОДГОТОВКА ДАННЫХ ДЛЯ ФИНАЛЬНОЙ СВОДКИ
# (Разместите этот блок кода прямо перед разделом "11. ФИНАЛЬНЫЙ СБОР РЕЗУЛЬТАТОВ")
# =============================================================================

# 1. Создаем пустой датафрейм для результатов мета-регрессии (если он еще не создан)
if (!exists("all_moderators_results_df")) {
  all_moderators_results_df <- data.frame(
    Moderator = character(),
    Outcome = character(),
    Coefficient = numeric(),
    p_value = numeric(),
    R_squared = numeric(),
    stringsAsFactors = FALSE
  )
  cat("Создан пустой датафрейм для результатов мета-регрессии.\n")
}

# 2. Создаем матрицу корреляций для ключевых метрик
# Выбираем только числовые переменные для корреляции
numeric_vars <- c("sens", "spec", "N_Total", "Test_Size", "Sample_Size", "Year")
numeric_data <- my_data_clean[numeric_vars]

# Удаляем строки с пропущенными значениями
numeric_data_clean <- numeric_data[complete.cases(numeric_data), ]

# Проверяем, достаточно ли данных для анализа корреляции
if (nrow(numeric_data_clean) >= 3) {
  # Вычисляем матрицу корреляций (используем метод Спирмена для устойчивости к ненормальности)
  correlation_matrix <- cor(numeric_data_clean, method = "spearman")
  cat("Матрица корреляций успешно создана.\n")
} else {
  # Если данных недостаточно, создаем пустую матрицу
  correlation_matrix <- matrix(NA, nrow = length(numeric_vars), ncol = length(numeric_vars))
  rownames(correlation_matrix) <- colnames(correlation_matrix) <- numeric_vars
  cat("ВНИМАНИЕ: Недостаточно данных для расчета значимой корреляционной матрицы.\n")
}

# 3. Дополнительная диагностика данных
cat("\nДИАГНОСТИКА ПЕРЕД ФИНАЛЬНОЙ СВОДКОЙ:\n")
cat("Размерность матрицы корреляций:", dim(correlation_matrix), "\n")
cat("Размер датафрейма мета-регрессии:", dim(all_moderators_results_df), "\n")

Матрица корреляций успешно создана.

ДИАГНОСТИКА ПЕРЕД ФИНАЛЬНОЙ СВОДКОЙ:
Размерность матрицы корреляций: 6 6 
Размер датафрейма мета-регрессии: 0 5 
